In [149]:
#import necessary library
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

In [ ]:
#fetch data
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
data = requests.get(url)
soup = BeautifulSoup(data.text, 'html5lib')

In [ ]:
#wrangling data
table_contents = []
table = soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text == 'Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = row.span.text.split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)
    
df = pd.DataFrame(table_contents)
df

In [ ]:
#clean some complex borough
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [ ]:
df.shape

In [ ]:
'''
#geocoder was unreliable
import geocoder
for row in df.itertuples():
    
    # initialize your variable to None
    lat_lng_coords = None
    latitudes = []
    longitudes = []

    # loop until you get the coordinates
    postal_code = getattr(row, 'PostalCode')
    while(lat_lng_coords is None):
      g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
      lat_lng_coords = g.latlng

    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    
    latitudes.append(latitude)
    longitudes.append(longitude)
'''

In [ ]:
locations = pd.read_csv('Geospatial_Coordinates.csv')
locations.head()

In [ ]:
df_locations = df.join(locations.set_index('Postal Code'), on='PostalCode')

In [ ]:
df_locations

In [ ]:
#Analysis the data by using k-means
#define foursquare api parameters
CLIENT_ID = 'W4X1V4JRMQTWAOZZGMF43JFAB5KJUZEGEE3ABFFD1BSOLV2L' # your Foursquare ID
CLIENT_SECRET = 'RGVN4O3VJEQ3UIME0I3WXP12B4DJBXPPIHASVR3KRVN0BV2G' # your Foursquare Secret
ACCESS_TOKEN = 'E3F4JNMGKRISHNCKVD5400TMI3L2HDQ1SZB1D2HZYFUBDHQS' # your FourSquare Access Token

VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
#Find the boroughs contained word 'Toronto'
toronto_data = df_locations[df_locations.Borough.str.contains('Toronto')]

In [ ]:
#get toronto-related boroughs' venues
toronto_venues = getNearbyVenues(toronto_data.Neighborhood, toronto_data.Latitude, toronto_data.Longitude)

In [ ]:
toronto_venues.shape

In [ ]:
toronto_venues['Venue Category'].nunique()

In [ ]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
idx = toronto_onehot.columns.tolist().index('Neighborhood')
fixed_columns = ['Neighborhood'] + list(toronto_onehot.columns[:idx]) + list(toronto_onehot.columns[idx+1:])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

In [ ]:
toronto_onehot.shape

In [ ]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

In [ ]:
#start k-means 
# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

In [ ]:
# add clustering labels
toronto_grouped.insert(0, 'Cluster Labels', kmeans.labels_)

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_data.join(toronto_grouped.set_index('Neighborhood'), on='Neighborhood')
toronto_merged = toronto_merged[toronto_merged.columns[:6]] #just get the first 6 columns
toronto_merged.head() # check the last columns!

In [ ]:
from geopy.geocoders import Nominatim
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

In [148]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

import folium # map rendering library
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters